In [2]:
plot_offline = True
import plotly

if plot_offline is False:
    # Start Plotting Online
    plotly.tools.set_credentials_file(username='herbzhaocam', api_key='Wvld9LkdNvnOPdifcjhE')
    plotly.tools.set_config_file(world_readable=False, sharing='private')
    import plotly.plotly as py

if plot_offline is True:
    plotly.offline.init_notebook_mode(connected=True) #required to run plotly in jupyter offline
    import plotly.offline as py
    
# for colours
import colorlover as cl
from IPython.display import HTML
# for plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff


import pandas as pd
import csv
import numpy as np
import scipy as sp
import math



# simulation components
import tm4.simClasses as sim
import matplotlib.pyplot as plt
import numpy as np
from tm4.colourTools import specToRGB



In [61]:
class chiral_nematic_simulation():
    def __init__(self):
        # counter for plotting
        self.item_number = 0
        self.traces = {}
        # simulation wavelength range
        self.wlRange = np.linspace(400,800,300)
        pass

    def materials(self):
        self.CNC = sim.UniaxialMaterial(1.586,1.524) # this might not be right
        self.air = sim.HomogeneousNondispersiveMaterial(1)
        self.cellulose = sim.HomogeneousNondispersiveMaterial(1.55)
        # Set the glass as a practical backend
        self.glass = sim.HomogeneousNondispersiveMaterial(1.55)
        self.front = sim.IsotropicHalfSpace(self.air)
        self.airhalf= sim.IsotropicHalfSpace(self.air)
        self.glasshalf = sim.IsotropicHalfSpace(self.glass)

        # set space before and after the sample
        self.s = sim.OptSystem()
        # set space before and after the sample
        self.s.setHalfSpaces(self.airhalf ,self.glasshalf)
        


    def simple_film(self):
        # helicoidal structure with CNC, pitch and thickness
        heli = sim.HeliCoidalStructure
        h1 = heli(self.CNC,180,1000)
        # set structure for simulation
        self.s.setStructure([h1])
        self.result = self.s.scanSpectrum(self.wlRange)
        plt.plot(self.result[0],self.result[1], color = specToRGB(self.result))
        # counter for plotting
        self.item_number += 1 
        

    def chiral_nematic_film(self, materials, half_pitch = 180, thickness = 1000):
        # helicoidal structure with CNC, pitch and thickness
        heli = sim.HeliCoidalStructure
        h1 = heli(materials, half_pitch, thickness)
        # set structure for simulation
        self.s.setStructure([h1])
        self.result = self.s.scanSpectrum(self.wlRange)
        plt.plot(self.result[0],self.result[1], color = specToRGB(self.result))
        # counter for plotting
        self.item_number += 1 

    def CNC_water_mixture_refractive_index(self, water_volume_fraction):
        water = 1.33
        CNC = (1.586, 1.524)
        # dielectric constant epsilon is sqrt of refractive index
        # for composite materials, dielectric constant calculation: 
        # e = e1*v1 + e2*v2  v is the volume fraction   http://www.sciencedirect.com/science/article/pii/S0030401899006951
        CNC_water_mixture  = (np.square(np.sqrt(CNC[0])*(1-water_volume_fraction)+np.sqrt(water)*water_volume_fraction),
                            np.square(np.sqrt(CNC[1])*(1-water_volume_fraction)+np.sqrt(water)*water_volume_fraction))

        self.CNC_water_mixture  = sim.UniaxialMaterial(CNC_water_mixture[0], CNC_water_mixture[1])
        return self.CNC_water_mixture
    
    
    def plot_spectrum(self):
        plt.rcParams['figure.figsize'] = (8,6)
        plt.rcParams['savefig.dpi'] = 100
        # change font size, axis  etc..
        plt.show()
        
    def normalisation_with_experiment_data(self, experiment_peak_intensity = 0.52, normalised_base_level = 0.1):
        # the highest peak (without any red shift)
        original_peak = self.chiral_nematic_film(self.CNC, self.original_half_pitch, self.original_half_pitch*self.pseudo_layers)
        simulation_maximum = max(self.result[1])
        self.normalisation_ratio = experiment_peak_intensity/simulation_maximum
        self.normalised_base_level = normalised_base_level
        
        
    def prepare_plotly_data(self, series_name):
        # nomalisation value
        normalised_y = np.asarray(self.result[1])*self.normalisation_ratio + self.normalised_base_level
        
        # this prepares the traces after every simulation
        self.traces[self.item_number] = go.Scatter(
            #self.traces[i] = go.Scattergl( # performs better at larger dataset
                x = self.result[0], #x value
                y = normalised_y, #y value
                mode = self.plot_type,
                name = series_name,
                #text='RH {}'.format(self.peak_data[sample_number]['humidity'][scan_number]), # hover text
                line = dict(
                    dash = self.dash_type, 
                    shape='spline',
                    # automatically assign colour based on peak
                    color = 'rgb({},{},{})'.format(specToRGB(self.result)[0],specToRGB(self.result)[1],specToRGB(self.result)[2])
                    ),
                 marker = dict(
                    size = 7*self.font_size_scale_factor,
                    #color=self.colour_scale[item_number],
                    color = 'rgba(0,0,0,0)',
                    opacity = 0.8, 
                    #symbol = self.marker_symbols[item_number], 
                    line = dict(  # outter rim of the marker,
                        width = 2, 
                        color = "black",
                        ),
                    ),
                showlegend = self.show_legend,
                )
        
    def plotly_plot_data(self):
        # this defines layout and plot the plotly thing
        # modify layout with dict()
        layout = dict(
            #title = '',
            xaxis= dict( 
                range = self.x_lim, # move the x_axis to start from 0.
                title= self.x_label,
                titlefont=dict(size=30*self.font_size_scale_factor, family = 'Times New Roman'),
                ticklen= 0,
                tickfont=dict(size=20*self.font_size_scale_factor, family = 'Times New Roman'),
                showline=True,
                zeroline= True,                
                gridwidth= 1,
                showgrid=True,
                ),
            yaxis = dict(
                range = self.y_lim,
                title = self.y_label,
                titlefont=dict(size=30*self.font_size_scale_factor, family = 'Times New Roman'),
                ticklen= 0,
                tickfont=dict(size=20*self.font_size_scale_factor, family = 'Times New Roman'),
                showline = True,
                zeroline = True,
                showgrid = True,
                gridwidth= 1,
                ),
            legend = dict(
                font=dict(size=15*self.font_size_scale_factor, family = 'Times New Roman'),
                xanchor=self.legend_anchor[0],
                yanchor=self.legend_anchor[1],
                x = self.legend_position[0],
                y = self.legend_position[1],
                # a box for legend
                bgcolor='white',
                #bordercolor='black',
                borderwidth=1,
                ), 
            margin = go.Margin(
                l=75*self.font_size_scale_factor,
                #r=50,
                #b=100,
                #t=100,
                #pad=5
                ),
            # transparent graph
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
            )

        
        # create data -- a list contains all the traces
        data = []
        for i in self.traces:
            data.append(self.traces[i])
        # create figure with data and layout
        fig = dict(data=data, layout=layout)
        # set export static image
        
        # generate the plot and save as svg
        a = py.iplot(fig,  filename='csv_plot', 
                     image_width = self.image_size[0],
                     image_height = self.image_size[1], 
                     image='svg',)


    
# initialise the class
chiral_simulation = chiral_nematic_simulation()
chiral_simulation.wlRange = np.linspace(400,800,400)
chiral_simulation.materials()


# Plotly settings
# axes
chiral_simulation.x_lim = ()
chiral_simulation.x_label = 'Wavelength (nm)'
chiral_simulation.y_lim = ()
chiral_simulation.y_label = 'Intensity (A.U.)'
# legend
chiral_simulation.show_legend = True
chiral_simulation.legend_position = (1, None)
chiral_simulation.legend_anchor = ('left','top') #x,y_anchor
# How to plot data? #'lines', 'markers', 'lines+markers'
chiral_simulation.plot_type = 'lines' #'lines', 'markers', 'lines+markers'
chiral_simulation.dash_type = None
# the font size factor
chiral_simulation.font_size_scale_factor = 1.5
# How large is the image (svg for aspect ratio and relative size to fonts)
chiral_simulation.image_size = (850,500)


#simulation starts here
#colour_change_mechanism = 'pitch_change_only'
#olour_change_mechanism = 'refractive_index_change_only'


# initialise parameters
pseudo_layers = 25
original_half_pitch = 140
chiral_simulation.pseudo_layers = pseudo_layers
chiral_simulation.original_half_pitch = original_half_pitch
chiral_simulation.normalisation_with_experiment_data(0.42,0.1)


for water_volume_fraction in [0,0.1,0.2,0.3,0.4]:
        # for swelling with both refractive index change and pitch change
        swollen_thickness = original_half_pitch*pseudo_layers/(1-water_volume_fraction)
        swollen_half_pitch = swollen_thickness/pseudo_layers
        CNC_water_mixture = chiral_simulation.CNC_water_mixture_refractive_index(water_volume_fraction)
        chiral_simulation.chiral_nematic_film(materials = CNC_water_mixture, half_pitch = swollen_half_pitch, thickness = swollen_thickness)
        chiral_simulation.prepare_plotly_data(series_name = water_volume_fraction)

        
colour_change_mechanism = 'pitch_change_only'    
chiral_simulation.dash_type = 'dash'
chiral_simulation.show_legend = False 
for water_volume_fraction in [0.1,0.2,0.3,0.4]:
        swollen_thickness = original_half_pitch*pseudo_layers/(1-water_volume_fraction)
        swollen_half_pitch = swollen_thickness/pseudo_layers
        chiral_simulation.chiral_nematic_film(materials = chiral_simulation.CNC, half_pitch = swollen_half_pitch, thickness = pseudo_layers*swollen_half_pitch)
        chiral_simulation.prepare_plotly_data(series_name = None)

        
# chiral_simulation.plot_spectrum() # plot in matplotlib
chiral_simulation.plotly_plot_data()




In [58]:
class chiral_nematic_simulation():
    def __init__(self):
        # counter for plotting
        self.item_number = 0
        self.traces = {}
        # simulation wavelength range
        self.wlRange = np.linspace(400,800,300)
        pass

    def materials(self):
        self.CNC = sim.UniaxialMaterial(1.586,1.524) # this might not be right
        self.air = sim.HomogeneousNondispersiveMaterial(1)
        self.cellulose = sim.HomogeneousNondispersiveMaterial(1.55)
        # Set the glass as a practical backend
        self.glass = sim.HomogeneousNondispersiveMaterial(1.55)
        self.front = sim.IsotropicHalfSpace(self.air)
        self.airhalf= sim.IsotropicHalfSpace(self.air)
        self.glasshalf = sim.IsotropicHalfSpace(self.glass)

        # set space before and after the sample
        self.s = sim.OptSystem()
        # set space before and after the sample
        self.s.setHalfSpaces(self.airhalf ,self.glasshalf)
        


    def simple_film(self):
        # helicoidal structure with CNC, pitch and thickness
        heli = sim.HeliCoidalStructure
        h1 = heli(self.CNC,180,1000)
        # set structure for simulation
        self.s.setStructure([h1])
        self.result = self.s.scanSpectrum(self.wlRange)
        plt.plot(self.result[0],self.result[1], color = specToRGB(self.result))
        # counter for plotting
        self.item_number += 1 
        

    def chiral_nematic_film(self, materials, half_pitch = 180, thickness = 1000):
        # helicoidal structure with CNC, pitch and thickness
        heli = sim.HeliCoidalStructure
        h1 = heli(materials, half_pitch, thickness)
        # set structure for simulation
        self.s.setStructure([h1])
        self.result = self.s.scanSpectrum(self.wlRange)
        plt.plot(self.result[0],self.result[1], color = specToRGB(self.result))
        # counter for plotting
        self.item_number += 1 

    def CNC_water_mixture_refractive_index(self, water_volume_fraction):
        water = 1.33
        CNC = (1.586, 1.524)
        # dielectric constant epsilon is sqrt of refractive index
        # for composite materials, dielectric constant calculation: 
        # e = e1*v1 + e2*v2  v is the volume fraction   http://www.sciencedirect.com/science/article/pii/S0030401899006951
        CNC_water_mixture  = (np.square(np.sqrt(CNC[0])*(1-water_volume_fraction)+np.sqrt(water)*water_volume_fraction),
                            np.square(np.sqrt(CNC[1])*(1-water_volume_fraction)+np.sqrt(water)*water_volume_fraction))

        self.CNC_water_mixture  = sim.UniaxialMaterial(CNC_water_mixture[0], CNC_water_mixture[1])
        return self.CNC_water_mixture
    
    
    def plot_spectrum(self):
        plt.rcParams['figure.figsize'] = (8,6)
        plt.rcParams['savefig.dpi'] = 100
        # change font size, axis  etc..
        plt.show()
        
    def normalisation_with_experiment_data(self, experiment_peak_intensity = 0.52, normalised_base_level = 0.1):
        # the highest peak (without any red shift)
        original_peak = self.chiral_nematic_film(self.CNC, self.original_half_pitch, self.original_half_pitch*self.pseudo_layers)
        simulation_maximum = max(self.result[1])
        self.normalisation_ratio = experiment_peak_intensity/simulation_maximum
        self.normalised_base_level = normalised_base_level
        
        
    def prepare_plotly_data(self, series_name):
        # nomalisation value
        normalised_y = np.asarray(self.result[1])*self.normalisation_ratio + self.normalised_base_level
        
        # this prepares the traces after every simulation
        self.traces[self.item_number] = go.Scatter(
            #self.traces[i] = go.Scattergl( # performs better at larger dataset
                x = self.result[0], #x value
                y = normalised_y, #y value
                mode = self.plot_type,
                name = series_name,
                #text='RH {}'.format(self.peak_data[sample_number]['humidity'][scan_number]), # hover text
                line = dict(
                    dash = self.dash_type, 
                    shape='spline',
                    # automatically assign colour based on peak
                    color = 'rgb({},{},{})'.format(specToRGB(self.result)[0],specToRGB(self.result)[1],specToRGB(self.result)[2])
                    ),
                 marker = dict(
                    size = 7*self.font_size_scale_factor,
                    #color=self.colour_scale[item_number],
                    color = 'rgba(0,0,0,0)',
                    opacity = 0.8, 
                    #symbol = self.marker_symbols[item_number], 
                    line = dict(  # outter rim of the marker,
                        width = 2, 
                        color = "black",
                        ),
                    ),
                showlegend = self.show_legend,
                )
        
    def plotly_plot_data(self):
        # this defines layout and plot the plotly thing
        # modify layout with dict()
        layout = dict(
            #title = '',
            xaxis= dict( 
                range = self.x_lim, # move the x_axis to start from 0.
                title= self.x_label,
                titlefont=dict(size=30*self.font_size_scale_factor, family = 'Times New Roman'),
                ticklen= 0,
                tickfont=dict(size=20*self.font_size_scale_factor, family = 'Times New Roman'),
                showline=True,
                zeroline= True,                
                gridwidth= 1,
                showgrid=True,
                ),
            yaxis = dict(
                range = self.y_lim,
                title = self.y_label,
                titlefont=dict(size=30*self.font_size_scale_factor, family = 'Times New Roman'),
                ticklen= 0,
                tickfont=dict(size=20*self.font_size_scale_factor, family = 'Times New Roman'),
                showline = True,
                zeroline = True,
                showgrid = True,
                gridwidth= 1,
                ),
            legend = dict(
                font=dict(size=15*self.font_size_scale_factor, family = 'Times New Roman'),
                xanchor=self.legend_anchor[0],
                yanchor=self.legend_anchor[1],
                x = self.legend_position[0],
                y = self.legend_position[1],
                # a box for legend
                bgcolor='white',
                #bordercolor='black',
                borderwidth=1,
                ), 
            margin = go.Margin(
                l=75*self.font_size_scale_factor,
                #r=50,
                #b=100,
                #t=100,
                #pad=5
                ),
            # transparent graph
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
            )

        
        # create data -- a list contains all the traces
        data = []
        for i in self.traces:
            data.append(self.traces[i])
        # create figure with data and layout
        fig = dict(data=data, layout=layout)
        # set export static image
        
        # generate the plot and save as svg
        a = py.iplot(fig,  filename='csv_plot', 
                     image_width = self.image_size[0],
                     image_height = self.image_size[1], 
                     image='svg',)


    
# initialise the class
chiral_simulation = chiral_nematic_simulation()
chiral_simulation.wlRange = np.linspace(400,950,400)
chiral_simulation.materials()


# Plotly settings
# axes
chiral_simulation.x_lim = ()
chiral_simulation.x_label = 'Wavelength (nm)'
chiral_simulation.y_lim = ()
chiral_simulation.y_label = 'Intensity (A.U.)'
# legend
chiral_simulation.show_legend = True
chiral_simulation.legend_position = (1, None)
chiral_simulation.legend_anchor = ('left','top') #x,y_anchor
# How to plot data? #'lines', 'markers', 'lines+markers'
chiral_simulation.plot_type = 'lines' #'lines', 'markers', 'lines+markers'
chiral_simulation.dash_type = None
# the font size factor
chiral_simulation.font_size_scale_factor = 1.5
# How large is the image (svg for aspect ratio and relative size to fonts)
chiral_simulation.image_size = (850,500)


#simulation starts here
#colour_change_mechanism = 'pitch_change_only'
#olour_change_mechanism = 'refractive_index_change_only'


# initialise parameters
pseudo_layers = 25
original_half_pitch = 200
chiral_simulation.pseudo_layers = pseudo_layers
chiral_simulation.original_half_pitch = original_half_pitch
chiral_simulation.normalisation_with_experiment_data(0.42,0.1)


for water_volume_fraction in [0,0.1,0.2,0.3,0.4,0.5]:
        # for swelling with both refractive index change and pitch change
        swollen_thickness = original_half_pitch*pseudo_layers/(1-water_volume_fraction)
        swollen_half_pitch = swollen_thickness/pseudo_layers
        CNC_water_mixture = chiral_simulation.CNC_water_mixture_refractive_index(water_volume_fraction)
        chiral_simulation.chiral_nematic_film(materials = CNC_water_mixture, half_pitch = swollen_half_pitch, thickness = swollen_thickness)
        chiral_simulation.prepare_plotly_data(series_name = water_volume_fraction)

        
colour_change_mechanism = 'pitch_change_only'    
chiral_simulation.dash_type = 'dash'
chiral_simulation.show_legend = False 
for water_volume_fraction in [0.1,0.2,0.3,0.4,0.5]:
        swollen_thickness = original_half_pitch*pseudo_layers/(1-water_volume_fraction)
        swollen_half_pitch = swollen_thickness/pseudo_layers
        chiral_simulation.chiral_nematic_film(materials = chiral_simulation.CNC, half_pitch = swollen_half_pitch, thickness = pseudo_layers*swollen_half_pitch)
        chiral_simulation.prepare_plotly_data(series_name = None)

        
# chiral_simulation.plot_spectrum() # plot in matplotlib
chiral_simulation.plotly_plot_data()


